# Projet : **Breast Cancer Detection using Deep Learning**




**This project aims to build a medical image classification model to distinguish between benign and malignant breast cancer cases using histopathological images.**
It is based on a Convolutional Neural Network (CNN) architecture trained on a labeled dataset.
The goal is to support medical diagnosis in an automated and reliable manner.


**Application Prototyping with Gradio**
To facilitate testing and demonstrations, we used Gradio to create an interactive interface.
Gradio allows for the creation of a simple user interface where a doctor or user can upload an image and instantly receive a prediction (benign or malignant).




let's import the necessary libraries :

<a name="1"></a>
## 1 - Packages

First, let's run the cell below to import all the packages you will need throughout this work.
- [Gradio](https://www.gradio.app/)  is the core package for building user interfaces in Python.
- [tensorflow](https://www.tensorflow.org/tutorials?hl=fr) is a popular library for building and training machine learning models in Python.
- [keras](https://keras.io/) is an open-source library, originally developed by Intel, specialized in computer vision and image processing.

In [1]:
import gradio as gr
import numpy as np
import tensorflow as tf
import cv2
import os

In [13]:
from google.colab import files
uploaded = files.upload()

Saving CM_weights-010-0.3063.hdf5 to CM_weights-010-0.3063.hdf5


In [15]:
# Chemin vers le fichier dans /content
model_path = "/content/CM_weights-010-0.3063.hdf5"

# Vérifie si le fichier existe
if os.path.exists(model_path):
    model = tf.keras.models.load_model(model_path)
    print("Modèle chargé avec succès.")
else:
    print(f"Fichier non trouvé à : {model_path}")

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.11/dist-packages/keras/src/optimizers/base_optimizer.py:86: UserWarning: Argument `decay` is no longer supported and will be ignored.
  warnings.warn(


Modèle chargé avec succès.


In [21]:
import gradio as gr
import numpy as np
import cv2
from PIL import Image

# Fonction de prédiction pour une seule image
def predict(file):
    # Ouvrir l’image avec PIL, puis convertir en tableau numpy
    img = Image.open(file).convert("RGB")
    image = np.array(img)

    # Redimensionner à 48x48
    image_resized = cv2.resize(image, (48, 48))

    # Normaliser
    image_resized = image_resized.astype("float32") / 255.0

    # Ajouter une dimension batch
    image_batch = np.expand_dims(image_resized, axis=0)

    # Prédire
    benign, malignant = model.predict(image_batch)[0]

    # Déterminer le label
    label = "benign" if benign > malignant else "malignant"
    confidence = max(benign, malignant)

    # Afficher sur l'image
    display_image = cv2.putText(
        image.copy(),
        f"{label}: {confidence*100:.2f}%",
        (10, 40),
        cv2.FONT_HERSHEY_SIMPLEX,
        1.2,
        (0, 255, 0) if label == "benign" else (0, 0, 255),
        3
    )

    # Convertir image BGR -> RGB
    display_image = cv2.cvtColor(display_image, cv2.COLOR_BGR2RGB)

    return display_image, f"{label}: {confidence*100:.2f}%", {label: float(confidence)}

In [22]:
# Interface Gradio avec une seule image en entrée
demo = gr.Interface(
    fn=predict,
    inputs=gr.File(label="Upload a Histopathological Image", file_types=["image"], file_count="single"),
    outputs=[
        gr.Image(label="Prediction Result"),
        gr.Textbox(label="Label"),
        gr.Label(num_top_classes=2, label="Confidence Score")
    ],
    title="Breast Cancer Detection using deep learning (Benign vs Malignant)",
    description="Upload a histopathological image to predict whether it is benign or malignant.",
    allow_flagging="never"
)

demo.launch()

/usr/local/lib/python3.11/dist-packages/gradio/interface.py:416: UserWarning: The `allow_flagging` parameter in `Interface` is deprecated.Use `flagging_mode` instead.
  warnings.warn(


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://f7d085cc3716b80527.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
